In [1]:
import pandas as pd
import os

# Folder and file setup
data_folder = 'data/'
input_file = 'mergedportail.xlsx'

# Read the merged Excel file into DataFrame (load all columns as is)
file_path = os.path.join(data_folder, input_file)
df = pd.read_excel(file_path, engine='openpyxl')

print(f"DataFrame shape: {df.shape}")
df.head()


DataFrame shape: (262636, 54)


,order_oc_id,customer_id,project_id,country_of_delivery,supply_center_id,order_type,direct_delivery,warehouse_id,country_of_origin,order_priority_type,...,order_description,order_completion,Total_LeadTime,order_weight_kg,order_volume_dm3,order_volume_m3,price_orderline,unique_order_code,unique_shipment_code,unique_backorder_code
0,18/5035/CH/NG111,CH001MCH,NG111MCH,NG,MSFL,med,No,BDX,FR,Emergency,...,EMERG MED OR NGALA (thru CM),Complete,81.0,0.29,2.80,0.00280,477.56,"NG111MCH,2018-11-23","NG,2019-01-04","18/5035/CH/NG111,SINSNEIOKN12,2019-01-04"
1,18/5035/CH/NG111,CH001MCH,NG111MCH,NG,MSFL,med,No,BDX,FR,Emergency,...,EMERG MED OR NGALA (thru CM),Complete,69.0,6.40,52.00,0.05200,377.60,"NG111MCH,2018-11-23","NG,2018-12-07","18/5035/CH/NG111,SINSIVCST18W1,2018-12-07"
2,18/5035/CH/NG111,CH001MCH,NG111MCH,NG,MSFL,med,No,BDX,FR,Emergency,...,EMERG MED OR NGALA (thru CM),Complete,69.0,0.40,3.25,0.00325,23.60,"NG111MCH,2018-11-23","NG,2018-12-07","18/5035/CH/NG111,SINSIVCST18W1,2018-12-07"
3,18/5035/CH/NG111,CH001MCH,NG111MCH,NG,MSFL,med,No,BDX,FR,Emergency,...,EMERG MED OR NGALA (thru CM),Complete,557.0,0.30,2.00,0.00200,15.70,"NG111MCH,2018-11-23","NG,2019-12-13","18/5035/CH/NG111,DINJTRAM1A-,2019-12-13"
4,18/5035/CH/NG111,CH001MCH,NG111MCH,NG,MSFL,med,No,BDX,FR,Emergency,...,EMERG MED OR NGALA (thru CM),Complete,557.0,0.60,4.00,0.00400,31.40,"NG111MCH,2018-11-23","NG,2019-12-13","18/5035/CH/NG111,DINJTRAM1A-,2019-12-13"


In [3]:
# ✅ Filter for South Sudan (SS)
df_south_sudan = df[df['country_of_delivery'] == 'CD'].reset_index(drop=True)

# ✅ Filter for Kenya (KE)
df_kenya = df[df['country_of_delivery'] == 'KE'].reset_index(drop=True)

# ✅ Filter for Nigeria (NG)
df_nigeria = df[df['country_of_delivery'] == 'NG'].reset_index(drop=True)

# ✅ Print shapes to verify
print(f"South Sudan (CD) DataFrame shape: {df_south_sudan.shape}")
print(f"Kenya (KE) DataFrame shape: {df_kenya.shape}")
print(f"Nigeria (NG) DataFrame shape: {df_nigeria.shape}")


South Sudan (CD) DataFrame shape: (34768, 54)
Kenya (KE) DataFrame shape: (14773, 54)
Nigeria (NG) DataFrame shape: (13971, 54)


In [ ]:
# Calculating the cost of binding capital - logic for risk premium to be implemented

# ✅ Set the interest rate (Swiss central bank rate)
interest_rate = 0.005  # 0.5% annual interest

# ✅ Calculate the capital binding cost per row based on invoiced_amount (per day)
df_south_sudan['capital_binding_cost'] = df_south_sudan['invoiced_amount'] * interest_rate / 365

# ✅ Check the result
print(df_south_sudan[['invoiced_amount', 'capital_binding_cost']].head())

# ✅ Optional: Calculate total capital binding cost for the dataset
total_binding_cost = df_south_sudan['capital_binding_cost'].sum()
print(f"Total Capital Binding Cost: {total_binding_cost:.2f}")


   invoiced_amount  capital_binding_cost
0             7.74              0.000106
1            98.60              0.001351
2          2417.04              0.033110
3           939.96              0.012876
4           456.00              0.006247
Total Capital Binding Cost: 280.46


In [ ]:
# Calculating the storage costs

# ✅ Warehouse and rent assumptions
annual_rent_chf = 25000  # CHF per year
warehouse_area_m2 = 500
warehouse_height_m = 3
utilization = 0.7

# ✅ Calculate usable warehouse volume (m³)
usable_volume_m3 = warehouse_area_m2 * warehouse_height_m * utilization

# ✅ Calculate rent per m³ per year and then per day
rent_per_m3_per_year = annual_rent_chf / usable_volume_m3
rent_per_m3_per_day = rent_per_m3_per_year / 365

print(f"Usable warehouse volume (m³): {usable_volume_m3}")
print(f"Rent per m³ per year (CHF): {rent_per_m3_per_year:.2f}")
print(f"Rent per m³ per day (CHF): {rent_per_m3_per_day:.4f}")

# ✅ Calculate the total volume occupied by each product (convert dm³ to m³)
df_south_sudan['volume_occupied_m3'] = (df_south_sudan['unit_volume'] / 1000) * df_south_sudan['quantity_ordered']

# ✅ Calculate the rent cost per day directly per product
df_south_sudan['rent_cost_per_day'] = df_south_sudan['volume_occupied_m3'] * rent_per_m3_per_day

# ✅ Preview results
print(df_south_sudan[['unit_volume', 'quantity_ordered', 'volume_occupied_m3', 'rent_cost_per_day']].head())

# ✅ Optional: Check the total allocated rent per day
total_rent_allocated_per_day = df_south_sudan['rent_cost_per_day'].sum()
print(f"Total Rent Allocated per Day (CHF): {total_rent_allocated_per_day:.2f}")


Usable warehouse volume (m³): 1050.0
Rent per m³ per year (CHF): 23.81
Rent per m³ per day (CHF): 0.0652
   unit_volume  quantity_ordered  volume_occupied_m3  rent_cost_per_day
0        0.018                20             0.00036           0.000023
1        0.022               200             0.00440           0.000287
2        0.196              3000             0.58800           0.038356
3        0.196              3000             0.58800           0.038356
4        0.004              3000             0.01200           0.000783
Total Rent Allocated per Day (CHF): 3107.38


In [ ]:
# Salary costs

scalable_staff = 10 * (utilization / 0.5)

# ✅ USD to CHF exchange rate (update as needed)
usd_to_chf = 0.90

# ✅ Average monthly salary assumptions (USD)
salary_bracket_1_usd = 3043/30  # Coordinators
salary_bracket_2_usd = 1275/30   # Supervisors / Specialists
salary_bracket_3_usd = 2100/30   # Activity Managers
salary_bracket_4_usd = 600/30    # Basic/Skilled (scalable)

# ✅ Convert to CHF
salary_bracket_1 = salary_bracket_1_usd * usd_to_chf
salary_bracket_2 = salary_bracket_2_usd * usd_to_chf
salary_bracket_3 = salary_bracket_3_usd * usd_to_chf
salary_bracket_4 = salary_bracket_4_usd * usd_to_chf

print(f"Bracket 1 (CHF): {salary_bracket_1:.2f}")
print(f"Bracket 2 (CHF): {salary_bracket_2:.2f}")
print(f"Bracket 3 (CHF): {salary_bracket_3:.2f}")
print(f"Bracket 4 (CHF): {salary_bracket_4:.2f}")


# Fixed team setup
fixed_cost = (1 * salary_bracket_1) + (2 * salary_bracket_2) + (2 * salary_bracket_3)

# Assume current warehouse utilization is known (0-1)
#current_utilization = total_volume_occupied_m3 / usable_volume_m3

# Scalable bracket 4 staff count
scalable_staff = 10 * (utilization / 0.5)
scalable_staff = max(0, scalable_staff)  # Ensure non-negative

# Scalable cost
scalable_cost = scalable_staff * salary_bracket_4

# Total monthly personnel cost for inventory management
total_personnel_cost = fixed_cost + scalable_cost


Bracket 1 (CHF): 2738.70
Bracket 2 (CHF): 1147.50
Bracket 3 (CHF): 1890.00
Bracket 4 (CHF): 540.00
